To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [31]:
import torch
# clear gpu cache
torch.cuda.empty_cache()

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\Aiden\AppData\Local\anaconda3\envs\finetune\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.5.2: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [5]:
from datasets import load_dataset
dataset = load_dataset("../../data/json/", num_proc=16)

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 57477
    })
})


We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [7]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset["train"])

Let's see how row 100 looks like!

In [8]:
dataset[100]

{'conversations': [{'content': 'You are an AI assistant that evaluates two responses to a prompt and determines which response is better or if it\'s a tie.\n\nPrompt:\n["[\\"Given 4 floats representing the elements of a quaternion, how can I identify which is the scalar component without knowing the convention beforehand? \\"]"]\n\nResponse A:\n["[\\"It\'s impossible to definitively identify the scalar component of a quaternion without knowing the convention beforehand. There are two common conventions for representing quaternions: scalar-first (w, x, y, z) and scalar-last (x, y, z, w). \\\\n\\\\nHowever, you can try to make an educated guess by analyzing the values of the elements. If one of the elements is significantly different from the others in magnitude, it might be the scalar component. However, this method is not foolproof and can lead to incorrect assumptions.\\\\n\\\\nThe best approach is to know the convention used for the quaternion beforehand or get this information from 

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [9]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [10]:
dataset[100]["text"]

'<bos><start_of_turn>user\nYou are an AI assistant that evaluates two responses to a prompt and determines which response is better or if it\'s a tie.\n\nPrompt:\n["[\\"Given 4 floats representing the elements of a quaternion, how can I identify which is the scalar component without knowing the convention beforehand? \\"]"]\n\nResponse A:\n["[\\"It\'s impossible to definitively identify the scalar component of a quaternion without knowing the convention beforehand. There are two common conventions for representing quaternions: scalar-first (w, x, y, z) and scalar-last (x, y, z, w). \\\\n\\\\nHowever, you can try to make an educated guess by analyzing the values of the elements. If one of the elements is significantly different from the others in magnitude, it might be the scalar component. However, this method is not foolproof and can lead to incorrect assumptions.\\\\n\\\\nThe best approach is to know the convention used for the quaternion beforehand or get this information from the s

In [11]:
print(dataset)

Dataset({
    features: ['conversations', 'text'],
    num_rows: 57477
})


In [12]:
# split the dataset into train, validation, and test using datasets

# Split into train (80%), validation (10%), test (10%)
dataset = dataset.train_test_split(test_size=0.4, train_size=0.6)
train_dataset = dataset["train"]
temp_dataset = dataset["test"]
temp_dataset = temp_dataset.train_test_split(test_size=0.5, train_size=0.5)
val_dataset = temp_dataset["train"]
test_dataset = temp_dataset["test"]
print(train_dataset)
print(val_dataset)
print(test_dataset)



Dataset({
    features: ['conversations', 'text'],
    num_rows: 34486
})
Dataset({
    features: ['conversations', 'text'],
    num_rows: 11495
})
Dataset({
    features: ['conversations', 'text'],
    num_rows: 11496
})


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [14]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 2,
        dataset_num_proc=1,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        eval_steps = 50,
        save_steps = 100,
        max_steps=500,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/34486 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/11495 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=32):   0%|          | 0/34486 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/11495 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [16]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><bos><start_of_turn>user\nYou are an AI assistant that evaluates two responses to a prompt and determines which response is better or if it\'s a tie.\n\nPrompt:\n["[\\"Hey, explain the first to third crusades for me\\"]"]\n\nResponse A:\n["[\\"The First Crusade took place in 1096 and was led by Godfrey of Bouillon, Robert the Pious, and William of l\'Estrange. The purpose of the Crusade was to recapture the Holy Land from Muslim hands and to promote the Christian cause. The Crusaders captured Jerusalem and established a Christian state in the city, which lasted until 1187.\\\\n\\\\nThe Second Crusade took place in 1147 and was led by Simon de Montfort. The aim of the Crusade was to reclaim the land from the hands of the Muslims and to strengthen the Christian hold on it. The\\\\u5341\\\\u5b57\\\\u519b returned to Jerusalem and captured it again, but they also faced significant losses and were eventually forced to retreat.\\\\n\\\\nThe Third Crusade took place in 1189 and was led 

Now let's print the masked out example - you should see only the answer is present:

In [17]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Response B is better.<end_of_turn>\n'

In [18]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4070 Ti. Max memory = 11.994 GB.
4.869 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 34,486 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: aidenyang66 (yyfsss) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.951100
2,4.152300
3,3.714200
4,2.215200
5,1.210400
6,0.385100
7,0.487400
8,0.553200
9,0.982900
10,0.575300


In [20]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2677.1557 seconds used for training.
44.62 minutes used for training.
Peak reserved memory = 6.191 GB.
Peak reserved memory for training = 1.322 GB.
Peak reserved memory % of max memory = 51.617 %.
Peak reserved memory for training % of max memory = 11.022 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [21]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, ...<end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [22]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down why the sky is blue! It's a beautiful example of a phenomenon called **Rayleigh scattering**. Here's the explanation:

**1. Sunlight and Colors:**

* Sunlight is actually made up of *all* the colors of the rainbow – red, orange, yellow


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [23]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3\\processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [2]:
if True:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "gemma-3", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "You are an AI assistant that evaluates two responses to a prompt and determines which response is better or if it's a tie.\n\nPrompt:\n[\"[\\\"What is the difference between marriage license and marriage certificate?\\\",\\\"How can I get both of them as quick as possible in California \\\",\\\"What is the minimal time to get them? 1 day or 10 days?\\\"]\"]\n\nResponse A:\n[\"[\\\"A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits.\\\",\\\"If you want to get both a marriage license and a marriage certificate in California as quickly as possible, there are a few steps you can follow:\\\\n\\\\n1.   Obtain a marriage license: You can obtain a marriage license from the county clerk's office in the county where you plan to marry. The process typically takes about 30 minutes to complete and you will need to bring identification, such as a driver's license or passport, and proof of your age, such as a birth certificate or a passport. The marriage license is valid for one year and there is no waiting period between the time you obtain the license and the time of the marriage.\\\\n2.   Get married: After obtaining the marriage license, you can get married at any location of your choice, as long as it is legally recognized as a place to get married.\\\\n3.   Obtain a marriage certificate: After the marriage has taken place, you need to obtain a marriage certificate from the same government agency that issued the marriage license. The process typically takes about 30 minutes to complete and you will need to bring the marriage license and a copy of the marriage certificate to the government agency.\\\\n\\\\nNote: You may need to check with your county clerk's office for the specific requirements and process for obtaining a marriage license and a marriage certificate in California.\\\",\\\"The minimal time to get a marriage license and a marriage certificate in California can vary depending on the specific requirements and process in your county. Generally, you can obtain a marriage license and a marriage certificate in California within a few days of applying, but it is best to check with your county clerk's office for the specific requirements and process for your county.\\\\n\\\\nIn some counties, you can obtain a marriage license and a marriage certificate on the same day, while in other counties, you may need to wait a few days. The process typically takes about 30 minutes to complete and you will need to bring identification, such as a driver's license or passport, and proof of your age, such as a birth certificate or a passport.\\\\n\\\\nIt is important to note that some counties in California require a waiting period between the time you obtain the marriage license and the time of the marriage. This waiting period can range from a few days to several weeks, and it is best to check with your county clerk's office for the specific requirements in your county.\\\"]\"]\n\nResponse B:\n[\"[\\\"A marriage license and a marriage certificate are two different legal documents that have separate purposes.\\\\n\\\\n1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the wedding ceremony takes place. The couple is required to meet certain criteria, such as being of a certain age or not being closely related. Once the license is issued, there's often a waiting period before the marriage ceremony can take place. The marriage license has to be signed by the couple, their witnesses, and the officiant conducting the marriage ceremony, then returned to the license issuer for recording.\\\\n\\\\n2. Marriage Certificate: A marriage certificate, on the other hand, is a document that proves a marriage has legally taken place. It's issued after the marriage ceremony, once the signed marriage license has been returned and recorded. The marriage certificate includes details about the couple, like their names, the date and location of their wedding, and the names of their witnesses. This document serves as the official record of the marriage and is often needed for legal transactions like changing a name, adding a spouse to insurance, or proving marital status.\\\",\\\"In California, here are the general steps to obtain a marriage license and marriage certificate as quickly as possible:\\\\n\\\\n1. Marriage License:\\\\n\\\\n   - Both parties must appear together at one of the County Clerk\\\\u2019s offices.\\\\n   - Complete a marriage license application before arriving (available online in many counties).\\\\n   - Bring valid identification (driver's license, passport, etc.) to prove your identity and age.\\\\n   - In California, you do not need to be a resident, and there's no waiting period or blood test required.\\\\n   - Pay the marriage license fee. The fee varies by county but is usually between $35 and $100.\\\\n   - The marriage license is issued immediately and is valid for 90 days.\\\\n\\\\n2. Marriage Ceremony: \\\\n\\\\n   - Arrange to have a wedding ceremony within 90 days of receiving the license. This could be a civil ceremony performed at the County Clerk\\\\u2019s office or a private ceremony.\\\\n   - The person performing the ceremony must return the completed license to the County Recorder\\\\u2019s office within 10 days of the ceremony.\\\\n\\\\n3. Marriage Certificate: \\\\n\\\\n   - After the signed marriage license has been returned and registered by the County Recorder, it becomes a marriage certificate.\\\\n   - You can order copies of the certificate for an additional fee. Processing times can vary, but expedited service may be available for an extra fee.\\\\n\\\\nRemember, each county may have slightly different rules and fees, so it's best to check with your specific County Clerk\\\\u2019s office for the most accurate information.\\\",\\\"The marriage license in California is issued immediately when you apply in person and meet all the requirements, so theoretically, you could get married on the same day if you have an officiant ready to perform the ceremony.\\\\n\\\\nHowever, the marriage certificate, which is the official record of your marriage, is only issued after the signed marriage license is returned and recorded by the county. The person who performs your marriage ceremony has 10 days to return the license to the county recorder's office.\\\\n\\\\nSo, while you could technically get married (i.e., obtain a license and have a ceremony) within a day, receiving the official marriage certificate will take a little longer, depending on how quickly the signed license is returned and how long the county recorder's office takes to process it.\\\"]\"]\n\nWhich response is better, or is it a tie? Please answer with \"Response A is better.\", \"Response B is better.\", or \"It's a tie.\".",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

==((====))==  Unsloth 2025.5.2: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
It's a tie.<end_of_turn>


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [25]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [26]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [27]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [28]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
